In [ ]:
import logging
import sys
import os

import numpy as np

In [ ]:
# https://jupylet.readthedocs.io/en/latest/index.html
import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

#from jupylet.audio.sample import Sample

In [ ]:
# game of life constants
gs = 16 # cols by rows
gw = 32 # size 
scr = [gs*gw,gs*gw] # screen

# matrix of size gs x gs
# create the initial matrix
ga = np.array([[np.random.randint(0,2) for c in range(gs)] for r in range(gs)])
#ga = np.array([[1 for c in range(gs)] for r in range(gs)])



In [ ]:
# function definition: this is the core part
def step(a,d):
    """Compute the result array"""
    # create a new empty array
    b = np.empty((d,d))
    # set list of neighbour indices: up,down,left,right
    nd = ((-1,0),(1,0),(0,-1),(0,1))
    # loop over elements
    for i in range(d):
        for j in range(d):
            sum = 0
            # loop over neighbours: this is the very core
            for dd in nd:
                    si = i + dd[0]
                    sj = j + dd[1]
                    # ignore boundary pixels
                    # if not (si < 0 or si >=d or sj < 0 or sj >= d):
                    #     sum += a[si,sj]
                    # alternatively, we can wrap at the boundaries
                    si = d-1 if si < 0 else 0 if si == d else si
                    sj = d-1 if sj < 0 else 0 if sj == d else sj
                    sum += a[si,sj]
            # !!!!!!! important !!!!!!!!!
            # !!! evaluate sum
            b[i,j] = 1 if sum == 2 else 0
            # !!!!!!! important !!!!!!!!!
    return b



In [ ]:
# init state

state = State(
    iters = 0,
    ga = ga,
    gb = np.empty((gs,gs)),
    left = False,
    dead = False,
    zombie = False,
    mouse = [0,0],
    mouse_set = [0,0,0]
)


app = App(width=scr[0] + gw, height=scr[1] + gw)#, log_level=logging.INFO)
background = '#444444'
foreground = '#eeeeee'
a0 = np.ones((gw, gw)) * 255
a2 = np.ones((app.height, app.width, 3)) * 255

items = np.array([[Sprite(a0, y=r*gw+gw,x=c*gw+gw) for c in range(gs)] for r in range(gs)])

field = Sprite(a2, y=app.height/2, x=app.width/2, color=background) 

info = Label(
    '0', font_size=16, color=foreground, 
    x=app.width//2, y=gw/2, 
    anchor_y='center', anchor_x='left'
    #font_path='fonts/PetMe64.ttf'
)


The two function arguments ct and dt will contain the current game time and the time since the function was last called (delta time). We can use these arguments to do interesting stuff, but you can ignore them for now.


In [ ]:
@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)
    
    field.draw()
    info.draw()
    for i in range(gs):
        for ii in range(gs):
            if state.ga[i,ii] > 0:
                items[i,ii].draw()
                #print(i,ii)
    


In [ ]:
@app.event
def key_event(key, action, modifiers):
        
    keys = app.window.keys
    
    if action == keys.ACTION_PRESS:
        
        if key == keys.LEFT:
            state.left = True


    if action == keys.ACTION_RELEASE:

    
        if key == keys.LEFT:
            state.left = False

            

In [ ]:
@app.event
def mouse_press_event(x, y, button):
    #state.mouse_set = (x,y,button)
    if button == 1:
        mx = (x - gw//2) // gw 
        my = (y - gw//2) // gw
        state.mouse_set = [1,mx,my]


In [ ]:
@app.run_me
def main_loop(ct,dt):
    state.iters = 0
    while True: # endless loop. We will break on certain conditions
        ct, dt = yield 1/30
        # check iterations
        if state.dead:
            #print("Dead")
            app.stop()
            break
        elif state.iters >= 100:
            #print("Still alive after ",state.iters," iterations")
            info.text = "Finished"
            app.stop()
            break



In [ ]:
@app.run_me_every(1/3)
def update(ct, dt):
    state.gb = state.ga.copy() # save old value
    state.ga = step(state.ga,gs).copy() # compute new values
    # insert mouse item, if exists
    if state.mouse_set[0] == 1:
        state.ga[state.mouse_set[2],state.mouse_set[1]] = 1
        state.mouse_set[0] = 0

    state.iters = state.iters + 1
    
    # check stady state
    if np.array_equal(state.gb,state.ga):
        if np.sum(state.ga) > 0:
            print("Zombie after ",state.iters," iterations")
            state.dead = True
            state.zombie = True
        else:
            print("Dead after ",state.iters," iterations")
            state.dead = True
        info.text = "Dead, Zombie: " + str(state.zombie)
    else:
        info.text = str(state.iters) # + f" - {state.mouse_set[0]},{state.mouse_set[1]},{state.mouse_set[2]}"


In [ ]:
app.run()
